# Demo 2: Simulated Annealing in Travelling Salesman Problem (TSP)

## Step 1: Import Libraries

In [13]:
from ctypes.wintypes import tagMSG

#Libraries: pandas, math, random
import pandas as pd
import math
import random

from networkx.algorithms.connectivity.edge_augmentation import collapse

## Step 2: Initialize Data

Read the .csv file to get the x and y coordinate of each city

In [6]:
df = pd.read_csv("./data/problemData.csv")
df.head()

,LocationName,LocationX,LocationY
0,Arad,91,492
1,Bucharest,400,327
2,Craiova,253,288
3,Drobeta,165,299
4,Eforie,562,293


Import the data into a dictionary, format is name:(x, y)

In [7]:
#create dictionary
cities = {}
#import data
cityName = df["LocationName"]
locationX = df["LocationX"]
locationY = df["LocationY"]
#zip the x and y coordinates into (x,y) format
location = zip(locationX, locationY)
#add to dictionary
for name, coordinate in zip(cityName, location):
    cities.update({name : coordinate})
#show
cities

{'Arad': (91, 492),
 'Bucharest': (400, 327),
 'Craiova': (253, 288),
 'Drobeta': (165, 299),
 'Eforie': (562, 293),
 'Fagaras': (305, 449),
 'Giurgiu': (375, 270),
 'Hirsova': (534, 350),
 'Iasi': (473, 506),
 'Lugoj': (165, 379),
 'Mehadia': (168, 339),
 'Neamt': (406, 537),
 'Oradea': (131, 571),
 'Pitesti': (320, 368),
 'Rimnicu': (233, 410),
 'Sibiu': (207, 457),
 'Timisoara': (94, 410),
 'Urziceni': (456, 350),
 'Vaslui': (509, 444),
 'Zerind': (108, 531)}

## Step 3: Functions

In order to find the best way, we need to calculate the distance first

In [11]:
#calculate the Euclidian distance
def distance(city1, city2):
    return math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)

With this function, we can calculate the total distance of a path (energy)

In [17]:
#calculate the total distance of a path
def energy(path, cities):
    dist = 0
    for i in range(len(path)):
        dist += distance(cities[path[i]], cities[path[(i + 1) % len(path)]])
    return dist

Generate some different solutions by exchanging two cities (neighbour)

In [15]:
def neighbour(path):
    neighbours = []
    for _ in range(5):
        tempPath = path[:]
        i, j = random.sample(range(len(path)), 2)
        tempPath[i], tempPath[j] = tempPath[j], tempPath[i]
        neighbours.append(tempPath)
    return neighbours

Define the schedule function

In [16]:
def schedule(k=20, lam=0.005, limit=10000):
    return lambda t: (k * math.exp(-lam * t) if t < limit else 0)

Core of simulated annealing

In [19]:
def simulatedAnnealing(cities, initialTemp = 20, coolingRate = 0.995, stopTemp = 1e-3, maxIter = 1000):
    currentPath = list(cities.keys())
    random.shuffle(currentPath)
    currentEnergy = energy(currentPath, cities)

    bestPath = currentPath[:]
    bestEnergy = currentEnergy

    temperate = initialTemp

    while temperate > stopTemp:
        for _ in range(maxIter):
            newPath = random.choice(neighbour(currentPath))
            newEnergy = energy(newPath, cities)
            deltaE = newEnergy - currentEnergy
            if deltaE < 0 or random.random() < math.exp(-deltaE / temperate):
                currentPath, currentEnergy = newPath, newEnergy
                if newEnergy < bestEnergy:
                    bestPath, bestEnergy = newPath[:], newEnergy
        temperate *= coolingRate
    return bestPath, bestEnergy

## Step 4: Run Simulated Annealing

In [20]:
bestPath, bestEnergy = simulatedAnnealing(cities)
print("Best path: ", bestPath)
print("Best energy: ", bestEnergy)

KeyboardInterrupt: 